In [1]:
import json
import pandas as pd

In [2]:
with open("df_all_brands_data.json", mode="r", encoding="utf-8") as f:
    data = json.load(f)
    f.close()

In [11]:
x = []
for i in data: # Loop through every page
    for j in i: # Loop through every individual car listing on a specific page
        x.append(j)

x = pd.DataFrame(x)


In [12]:
x

,marke,modell,variante,titel,form,fahrzeugzustand,leistung,getriebe,farbe,preis,kilometer,erstzulassung,fahrzeughalter,standort,url_to_crawl,page_rank,total_num_pages
0,Ferrari,458,,Bugatti Veyron Grand Sport|VITESSE SUSPENSION|...,Cabrio / Roadster,Unfallfrei,736 kW (1.001 PS),Automatik,,1.570.800 €,28.500 km,03/2015,2,DE-90439 Nürnberg,https://suchen.mobile.de/fahrzeuge/details.htm...,1,2
1,Ferrari,458,,Bugatti Veyron 16.4,Sportwagen / Coupé,Unfallfrei,736 kW (1.001 PS),Automatik,,1.350.000 €,22.900 km,11/2008,,DE-52477 Alsdorf / Aachen,https://suchen.mobile.de/fahrzeuge/details.htm...,1,2
2,Ferrari,458,,Bugatti Veyron,Cabrio / Roadster,,,Automatik,,2.866.585 €,571 km,01/2014,,GB-SM7 1AT Banstead,https://suchen.mobile.de/fahrzeuge/details.htm...,2,2
3,Ferrari,458,,Bugatti Chiron,Van / Minibus,Beschädigt,,,,9.999.999 €,100.000 km,08/1990,,DE-14532 Europarc-Dreilinden,https://suchen.mobile.de/fahrzeuge/details.htm...,2,2
